In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog
from scipy.linalg import lstsq
import matplotlib.pyplot as plt
from scipy.stats import norm
import math

## Helper functions

In [2]:
def L1(X,y, d):
    val = X.flatten()
    m = len(y)
    # Variables are: [a, b, c, slack_2, ..., slack_n]

    # Coefficients for the objective function
    c = np.zeros(m + d+1)
    c[3:] = 1  # We want to minimize the sum of slack variables

    # Constraints matrix A_ub and vector b_ub for the inequalities A_ub @ x <= b_ub
    A_ub = np.zeros((2 * m, m + d+1))
    b_ub = np.zeros(2 * m)

    for i in range(d, -1, -1):
        A_ub[:m, d-i] = np.power(val, i)
        A_ub[m:, d-i] = -np.power(val, i)

    A_ub[:m, d+1:] = -np.identity(m)
    A_ub[m:, d+1:] = -np.identity(m)

    b_ub[:m] = y
    b_ub[m:] = -y

    # Bounds for the variables: a, b are unbounded, slack variables are >= 0
    bounds = [(None, None)]*3 + [(0, None)] * m

    # Solving the linear programming problem
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds)

    return result.x[:d+1]

def L2(X,y, d):
# Reshape X to a column vector (if needed)
    val = X.flatten()
    m = len(y)

    A_ub = np.zeros((m, d+1))

    for i in range(d, -1, -1):
        A_ub[:, d-i] = np.power(val, i)

    result, _, _, _ = lstsq(A_ub, y)

    return result

def find_root(a,b,c):
    # Compute the discriminant
    discriminant = b**2 - 4*a*c

    # Case 1: No real roots (discriminant < 0)
    if discriminant < 0:
        return None, None

    # Case 2: One real root (discriminant = 0)
    elif discriminant == 0:
        root = -b / (2*a)
        return root, None

    # Case 3: Two real roots (discriminant > 0)
    else:
        root1 = (-b + math.sqrt(discriminant)) / (2*a)
        root2 = (-b - math.sqrt(discriminant)) / (2*a)
        return root1, root2

def calculateEmpiricalError(results, t, y):
    results = np.array(results)
    hypothesis_results = np.sign(results-t)

    sum = 0.0
    for j in range(len(hypothesis_results)):
        if y[j] != hypothesis_results[j]:
            sum += 1.0

    return sum/len(hypothesis_results)

def find_optimal_t(X, y, a_1, b_1, c_1, a_2, b_2, c_2):
    #calculate p_1,2(x^j) for all training point x
    p_1_pairs = [(a_1 * x*x + b_1*x+c_1, y_val) for x, y_val in zip(X, y)]
    p_2_pairs = [(a_2 * x*x + b_2*x+c_2, y_val) for x, y_val in zip(X, y)]

    #sort lists
    p_1_pairs_sorted = sorted(p_1_pairs, key=lambda pair: pair[0])
    p_2_pairs_sorted = sorted(p_2_pairs, key=lambda pair: pair[0])

    # Extract sorted p_1_results, p_2_results, and corresponding sorted y
    p_1_results_sorted = [pair[0] for pair in p_1_pairs_sorted]
    y_sorted_1 = [pair[1] for pair in p_1_pairs_sorted]

    p_2_results_sorted = [pair[0] for pair in p_2_pairs_sorted]
    y_sorted_2 = [pair[1] for pair in p_2_pairs_sorted]


    #initialize optimal t and empirical error
    t_1_opt = 0
    t_2_opt = 0

    curr_emp_err_1 = calculateEmpiricalError(p_1_results_sorted, t_1_opt, y_sorted_1)
    curr_emp_err_2 = calculateEmpiricalError(p_2_results_sorted, t_2_opt, y_sorted_2)

    #find t_1_opt by iterating through sorted p_1_results
    for i in range(1, len(X)):
        left = max(-1.0, p_1_results_sorted[i-1])
        right = min(1.0, p_1_results_sorted[i])

        if left <= right:
            curr_t_1 = (right + left) / 2.0
            next_emp_err_1 = calculateEmpiricalError(p_1_results_sorted, curr_t_1, y_sorted_1)

            if next_emp_err_1 < curr_emp_err_1:
                t_1_opt = curr_t_1
                curr_emp_err_1 = next_emp_err_1

    #find t_2_opt by iterating through sorted p_2_results
    for i in range(1, len(X)):
        left = max(-1.0, p_2_results_sorted[i-1])
        right = min(1.0, p_2_results_sorted[i])

        if left <= right:
            curr_t_2 = (right + left) / 2.0
            next_emp_err_2 = calculateEmpiricalError(p_2_results_sorted, curr_t_2, y_sorted_2)

            if next_emp_err_2 < curr_emp_err_2:
                t_2_opt = curr_t_2
                curr_emp_err_2 = next_emp_err_2
    return t_1_opt, t_2_opt

def calculate_error(a, root_1,root_2):
    left_err, right_err = 0.0,0.0
    if a > 0:
        if root_1 < -r:
            left_err += norm.cdf(-r)-norm.cdf(root_1)
        elif -r <= root_1 and root_1<= 0 :
            left_err += norm.cdf(root_1) - norm.cdf(-r)

        if r < root_2:
            right_err += 1-norm.cdf(root_2) + norm.cdf(r)-0.5
        elif 0 <= root_2 and root_2 <= r:
            right_err += 1-norm.cdf(r) + norm.cdf(root_2)-0.5
    elif a<0:
        if root_1 < -r:
            left_err += norm.cdf(root_1)
        elif -r <= root_1 and root_1<= 0 :
            left_err += norm.cdf(-r) +0.5-norm.cdf(root_1)

        if r < root_2:
            right_err += norm.cdf(root_2)-norm.cdf(r)
        elif 0 <= root_2 and root_2 <= r:
            right_err += norm.cdf(r)-norm.cdf(root_2)

    return left_err+right_err

In [3]:
# Function to compute empirical error for a given threshold t
def empirical_error(p_x, y, t):
    predictions = np.sign(p_x - t)
    return np.mean(predictions != y)
def foo(X, y, a, b, c):
    p_x = np.array([a*x*x+b*x+c for x in X])
    sorted_indices = np.argsort(p_x)
    #print(p_x.shape)
    print(sorted_indices.shape)
    x_sorted = X.flatten()[sorted_indices]
    y_sorted = y[sorted_indices]
    p_x_sorted = p_x[sorted_indices]


    # Compute the empirical error for t = -1
    best_t = -1
    min_error = empirical_error(p_x_sorted, y_sorted, best_t)

    # Evaluate midpoints between sorted polynomial values
    for i in range(len(p_x_sorted) - 1):
        t_candidate = (p_x_sorted[i] + p_x_sorted[i + 1]) / 2
        error = empirical_error(p_x_sorted, y_sorted, t_candidate)

        if error < min_error:
            min_error = error
            best_t = t_candidate

    # Check empirical error for t = 1 as well
    error_at_1 = empirical_error(p_x_sorted, y_sorted, 1)
    if error_at_1 < min_error:
        min_error = error_at_1
        best_t = 1

    return best_t


## Main

In [10]:
data = pd.read_csv("data.csv", header=None)
data = data.to_numpy()

points, sigma, r = data[0,0].astype('int64'), data[0,1], data[0,2]

y = data[1:,0]
X = data[1:,1]

#y = data["y"].to_numpy()
#data = data.drop(columns="y")
#X = data.to_numpy()
# print a few data samples
#print(data.head())
a_1,b_1,c_1 = L1(X,y, 2)
print("L1 coefficient: "+ str(a_1) + ", " + str(b_1) + ", " + str(c_1))
a_2,b_2, c_2 = L2(X,y, 2)
print("L2 coefficient: "+ str(a_2) + ", " + str(b_2) + ", " + str(c_2))

L1_root_1, L1_root_2 = find_root(a_1,b_1,c_1)
L2_root_1, L2_root_2 = find_root(a_2,b_2,c_2)

#t_1,t_2 = find_optimal_t(X, y, a_1, b_1, c_1, a_2, b_2, c_2)
#t_1, t_2 = 0.0,0.0
t_1, t_2 = find_optimal_t(X,y,a_1,b_1,c_1, a_2, b_2, c_2)
print(f"t1={t_1}, t2={t_2}")

L1_root_1, L1_root_2 = find_root(a_1,b_1,c_1-t_1)
L2_root_1, L2_root_2 = find_root(a_2,b_2,c_2-t_2)
print("L1 roots: " + str(L1_root_1) + ", " + str(L1_root_2))
print("L2 roots: " + str(L2_root_1) + ", " + str(L2_root_2))


L1 coefficient: 0.16104511746364447, -0.3700072019777397, -0.7992328479129627
L2 coefficient: 0.026823289348625413, -0.2977577602159688, -0.2310551677107437
t1=-0.24251783533472565, t2=0.1064814755378205
L1 roots: 3.334302607175727, -1.0367650735633354
L2 roots: 12.137482480481738, -1.0367648784763874


Depending on the roots, we have to calculate errors differently

In [11]:
#err_1 = norm.cdf(L1_root_1) + 0.5 - norm.cdf(-1.2815515655446004) + norm.cdf(L1_root_2)-norm.cdf(0.8416212335729143)
err_1 = norm.cdf(-1.0364333894937898) - norm.cdf(L1_root_2) + norm.cdf(0.6744897501960817)-0.5 + 1.0-norm.cdf(L1_root_1)
err_2 = norm.cdf(-1.0364333894937898) - norm.cdf(L2_root_2) + norm.cdf(0.6744897501960817)-0.5 + 1.0-norm.cdf(L2_root_1)
#err_2 = calculate_error(a_2, L2_root_1, L2_root_2)

print(f"err1={err_1}, err2={err_2}")

err1=0.25050488961393014, err2=0.2500772762882548
